In [124]:
import pandas as pd
import ast
import pyGM as gm
import numpy as np

In [125]:
cards = pd.read_csv('data/cards.csv')

In [126]:
cards['cmc'] = cards['manaCost'].apply(lambda x: ''.join(''.join(str(x).split('{')).split('}')))
for index, row in cards.iterrows():
    X = row['cmc']
    list(X)
    x_ = 0
    for x in X:
        try:
            x_+=int(x)
        except:
            x_+=1
    row['cmc']=x_

In [128]:
decks = dict()
decks['modern2018'] = pd.read_csv('data/deckListmodernMetaDecks2018.csv')
decks['modern2019'] = pd.read_csv('data/deckListmodernMetaDecks2019.csv')
decks['modern2020'] = pd.read_csv('data/deckListmodernMetaDecks2020.csv')


In [129]:
decklists = dict()
decklists['modern2020'] = decks['modern2020'][decks['modern2020']['1']=='Red Deck Wins']['3']
decklists['modern2019'] = decks['modern2019'][decks['modern2019']['1']=='Red Deck Wins']['3']
decklists['modern2018'] = decks['modern2018'][decks['modern2018']['1']=='Red Deck Wins']['3']


In [130]:
decklists['modern2020'] = decklists['modern2020'].to_dict()[0]
decklists['modern2020'] = ast.literal_eval(decklists['modern2020'])
decklists['modern2019'] = decklists['modern2019'].to_dict()[0]
decklists['modern2019'] = ast.literal_eval(decklists['modern2019'])
decklists['modern2018'] = decklists['modern2018'].to_dict()[1]
decklists['modern2018'] = ast.literal_eval(decklists['modern2018'])

In [131]:
simMain = dict()
simSide = dict()
for D in decklists:
    for i,deck in enumerate(decklists[D]):
        for ty in deck:
            if ty=='SIDEBOARD':
                sim=simSide
            else:
                sim=simMain
            for card in deck[ty]:
                card = card.split(' ')
                freq = card[0]
                name = ' '.join(card[1:])
                if name not in sim:
                    sim[name] = list()
                    sim[name].append([i,int(freq)])
                else:
                    isIn = False
                    for i_,freq_ in sim[name]:
                        if i_==i:
                            isIn=True
                            freq_+=int(freq)
                    if isIn==False:
                        sim[name].append([i,int(freq)])


In [137]:
sim=simSide
d = pd.DataFrame(columns=columns)
for i in range(190):
    d.loc[i] = 0.0
for card in sim:
    for deck,freq in sim[card]:
        d.loc[deck][card]=freq
        
corr = d.corr()
for index, row in corr.iterrows():
    for i,r in enumerate(row):
        try:
            int(r)
        except:
            row[i]=-1
corrSide=corr